In [79]:
import pandas as pd

prediction_year = 2022
df1 = pd.read_parquet(f'../actuals_new/cm/window=Y{prediction_year}/cm_actuals_{prediction_year}.parquet')
df2 = pd.read_parquet(f'../actuals_preliminary/cm/window=Y{prediction_year}/cm_actuals_{prediction_year}.parquet')
# reset index
df1.reset_index(drop=False, inplace=True)
df2.reset_index(drop=False, inplace=True)
# df1 = df1[['month_id', 'country_id', 'outcome']]
# df2 = df2[['month_id', 'country_id', 'ged_sb']]
# df1
# df2
df1.rename(columns={'outcome': 'ged_sb'}, inplace=True)

In [80]:
df1

,month_id,country_id,ged_sb
0,505,1,0
1,505,2,0
2,505,3,0
3,505,4,0
4,505,5,0
...,...,...,...
2287,516,242,0
2288,516,243,0
2289,516,244,0
2290,516,245,3


In [81]:
# Merge DataFrames on 'month_id' and 'country_id' with outer join to handle missing rows
merged_df = pd.merge(df1, df2, on=['month_id', 'country_id'], how='outer', suffixes=('_new', '_old'))

# Identify rows where 'ged_sb' values differ or are missing in one DataFrame
differences_df = merged_df[merged_df['ged_sb_new'] != merged_df['ged_sb_old']]
# calculate the difference between the two values
differences_df['ged_sb_diff'] = differences_df['ged_sb_new'] - differences_df['ged_sb_old']
# drop 'index'
differences_df.drop(columns=['index'], inplace=True)

# print sum of differences
print("Absolute sum of differences: ", abs(differences_df['ged_sb_diff']).sum())
# Display the result
differences_df

Absolute sum of differences:  249199.0


/var/folders/88/v5lhf14x1r5bv01kg4dwr5780000gn/T/ipykernel_10300/734824422.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  differences_df['ged_sb_diff'] = differences_df['ged_sb_new'] - differences_df['ged_sb_old']
/var/folders/88/v5lhf14x1r5bv01kg4dwr5780000gn/T/ipykernel_10300/734824422.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  differences_df.drop(columns=['index'], inplace=True)


,month_id,country_id,ged_sb_new,ged_sb_old,ged_sb_diff
15,505,17,0,1.0,-1.0
44,505,47,90,252.0,-162.0
47,505,50,93,95.0,-2.0
50,505,54,0,3.0,-3.0
52,505,56,1,0.0,1.0
...,...,...,...,...,...
2251,516,165,19,0.0,19.0
2253,516,167,406,67.0,339.0
2278,516,220,116,105.0,11.0
2285,516,235,23,21.0,2.0


In [82]:
# amount of country_ids in df1 vs df2
print(len(df1['country_id'].unique()))
print(len(df2['country_id'].unique()))

# print difference in country_ids
print(set(df1['country_id'].unique()) - set(df2['country_id'].unique()))

# check if amount of months available per each country_id is the same
df1_months_per_country = df1.groupby('country_id').count()
df2_months_per_country = df2.groupby('country_id').count()

df1_months_per_country = df1_months_per_country.reset_index()
df2_months_per_country = df2_months_per_country.reset_index()
df1_months_per_country[["country_id", "month_id"]] == df2_months_per_country[["country_id", "month_id"]]
# assert all true
print("All the same countries and months are present:")
print((df1_months_per_country[["country_id", "month_id"]] == df2_months_per_country[
    ["country_id", "month_id"]]).all().all())



191
191
set()
All the same countries and months are present:
True
